In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# <center>Mobile Price Classification </center>
This dataset contains the sales data of mobile phones of various companies. Need to find out relation between various features of mobile phone and it's selling price.<br>
In this problem we need to predict the price range considering the mobile phone features.

In [ ]:
#Toggle code - button
from IPython.display import HTML,Image
HTML('''
<script>
code_show=true;
function code_toggle(){
if (code_show){
$('div.input').hide();
document.getElementById('btn_toggle').value='Show Code';
}else{
$('div.input').show();
document.getElementById('btn_toggle').value='Hide Code';
}
code_show=!code_show
}
</script>
<style>
.output_png {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
    }
</style>
<form action="javascript:code_toggle()">
<input style = "float:right" type="submit" id="btn_toggle" value='Hide Code'>
'''
)

In [ ]:
#import required libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
import mlxtend
from mlxtend.preprocessing import standardize
from sklearn.metrics import classification_report, confusion_matrix,roc_auc_score,auc,roc_curve
from sklearn.model_selection import train_test_split

In [ ]:
#Visualization settings
sns.set_style(style='white')
sns.set(rc={
    'figure.figsize': (12,7),
    'axes.facecolor': 'white',
    'axes.grid': True,
    'grid.color': '.9',
    'axes.linewidth': 1.0,
    'grid.linestyle': u'-'},
    font_scale=1.5)
custom_colors=["#3498db", "#95a5a6","#34495e", "#2ecc71", "#e74c3c"]
sns.set_palette(custom_colors)

In [ ]:
import os
!ls ../input/

In [ ]:
Image("../input/smartphones-image/smartphones.jpg", width=500, height=100)

In [ ]:
df_train=pd.read_csv("../input/mobile-price-classification/train.csv")
print ("Training data loaded")

# <center> Dataset description </center>

In [ ]:
print ("Sample data frame: \n")
df_train.head(3)

In [ ]:
print ("Features in the dataset :\n")
for col in df_train.columns:
    print (col)
print ("\nTarget column is : price_range")
print ("\nShape of the dataframe: ", df_train.shape)

In [ ]:
df_train.info()

<h3 style='background-color:yellow'>
20 Features and 1 Target column.<br>
19 integer, 2 float columns.
</h3>

In [ ]:
print ("Null values in features: \n")
df_train.isnull().sum()

<h3 style='background-color:yellow'>
No Null values in the dataset.
</h3>

## <center> Features </center>

### 1. battery_power
continuous variable

In [ ]:
print (df_train.battery_power.describe())
plt.figure(figsize=(20,10))
plt.subplot(2,2,1)
sns.distplot(df_train.battery_power);


<h3 style='background-color:yellow'>
Battery power is a continuous variable
</h3>

### 2.  blue
Has bluetooth or not.
Categorical variable

In [ ]:
print ("Unique values :")
df_train.blue.unique()

In [ ]:
df_train.blue.value_counts().plot.bar();
plt.title("Blue -distribution");

<h3 style='background-color:yellow'>
blue is a categorical variable.
</h3>

### 3. clock speed

In [ ]:
df_train.clock_speed.describe()

In [ ]:
plt.figure(figsize=(20,10));
plt.suptitle("Clock speed distribution");
plt.subplot(2,2,2)
sns.distplot(df_train.clock_speed).set_title("Clock speed distribution");
plt.subplot(2,2,1)
df_train.clock_speed.value_counts().plot.bar();

<h3 style='background-color:yellow'>
clock speed is a continuous variable.
</h3>

### 4. Dual sim

In [ ]:
df_train.dual_sim.value_counts().plot.bar().set_title("Dual sim - distribution");

<h3 style='background-color:yellow'>
Dual sim is categorical variable.
</h3>

### 5. Fc
Front camera megapixels

In [ ]:
df_train.fc.value_counts().plot.bar().set_title("Fc- Distribution");

<h3 style='background-color:yellow'>
Front camera pixels is categorical variable.
</h3>

### 6. Four G

In [ ]:
df_train.four_g.value_counts().plot.bar();

<h3 style='background-color:yellow'>
Four G is categorical variable.
</h3>

### 7. Int memory

In [ ]:
print ("Unique values: \n", np.sort(df_train.int_memory.unique()))

In [ ]:
print ("Value description: \n",df_train.int_memory.describe())

In [ ]:
plt.figure(figsize=(25,10))
plt.subplot(2,2,1)
sns.distplot(df_train.int_memory)
plt.subplot(2,2,2)
df_train.int_memory.value_counts().plot.bar();
plt.suptitle("Int.memory -distribution");

<h3 style='background-color:yellow'>
Int. memory is categorical variable.
</h3>

### 8. M Dep
Mobile Depth in cm

In [ ]:
df_train.m_dep.value_counts().plot.bar();
plt.title("M dep - distribution");

<h3 style='background-color:yellow'>
M Dep is continuous variable.
</h3>

### 9. Mobile wt

In [ ]:
df_train.mobile_wt.describe()

In [ ]:
sns.distplot(df_train.mobile_wt);

<h3 style='background-color:yellow'>
Mobile wt. is continuous variable.
</h3>

### 10. N cores
Number of cores of the processor

In [ ]:
sns.countplot(data=df_train, x="n_cores", order=df_train.n_cores.value_counts().index);

<h3 style='background-color:yellow'>
N cores is categorical variable.
</h3>

### 11.Pc 
Primary Camera Megapixes

In [ ]:
sns.countplot(data=df_train,x="pc",order=df_train.pc.value_counts().index);

In [ ]:
print ("Unique values : \n")
df_train.pc.unique()

<h3 style='background-color:yellow'>
Primary camera mega pixels is categorical variable.
</h3>

### 12. Price Range
This will be target category.

In [ ]:
print ("Unique values: \n")
df_train.price_range.unique()

In [ ]:
sns.countplot(data=df_train,x="price_range");

<h3 style='background-color:yellow'>
All price ranges got equal distributions. <br>
Price range is a categorical variable.    
</h3>

### 13. Px Height
Pixel Resolution Height

In [ ]:
df_train.px_height.describe()

In [ ]:
sns.distplot(df_train.px_height);

In [ ]:
print ("Observations having zero pixel resolution :\n")
df_train[df_train["px_height"]==0]

<h3 style='background-color:yellow'>
2 records having pixel height = 0 doesn't make sense. <br>
As the pixel height increases , number of observations gradualy reduces. <br>
    Px Height is continuous variable.
</h3>

### 14. Px Width
Pixel resolution width

In [ ]:
df_train.px_width.describe()

In [ ]:
sns.distplot(df_train.px_width);

<h3 style='background-color:yellow'>
Px. Width is categorical variable.
</h3>

### 15. Ram

In [ ]:
df_train.ram.describe()

In [ ]:
sns.distplot(df_train.ram);

<h3 style='background-color:yellow'>
Ram is continuous variable.
</h3>

### 16.Sc H
Screen height of mobile in CM.

In [ ]:
print ("Unique values: \n")
np.sort(df_train.sc_h.unique())

In [ ]:
sns.countplot(df_train.sc_h);

<h3 style='background-color:yellow'>
Screen height is continuous variable.
</h3>

### 17. Sc.W
Screen width of mobile in CM

In [ ]:
print ("Unique values : \n")
np.sort(df_train.sc_w.unique())

In [ ]:
sns.countplot(df_train.sc_w).set_title("Screen width -distribution");

<h3 style='background-color:yellow'>
Screen width is continuous variable.
</h3>

### 18. Talk time
Maximum talking time for single charge

In [ ]:
df_train.talk_time.describe()

In [ ]:
sns.countplot(df_train.talk_time);
plt.title("Talk time - distribution");

<h3 style='background-color:yellow'>
Talk time is continuous variable.
</h3>

### 19. Three G

In [ ]:
print ("Unique values: \n")
df_train.three_g.unique()

In [ ]:
plt.figure(figsize=(15,10))
plt.subplot(2,2,1)
sns.countplot(df_train.three_g);
plt.subplot(2,2,2)
print ("Value counts for Three G: ",np.unique(df_train.three_g,return_counts=True))
plt.pie(df_train.three_g.value_counts(),autopct="%.2f",colors=['green','red'],labels=["Three G","Not Three G"]);
plt.suptitle("Three G - distribution");

<h3 style='background-color:yellow'>
Three G is discrete variable.
</h3>

### 20. Touch screen

In [ ]:
print ("Value counts: \n")
df_train.touch_screen.value_counts()

In [ ]:
plt.pie(df_train.touch_screen.value_counts(),autopct="%.2f",colors=['green','red'],labels=["Touch screen", "Normal screen"]);
plt.title("Touch screen distribution");

<h3 style='background-color:yellow'>
Touch screen and normal screen are almost same distribution in the dataset. <br>
    Is a discrete variable.
</h3>

### 21. Wifi

In [ ]:
plt.pie(df_train.wifi.value_counts(),autopct="%.2f",colors=['green','red'],labels=["Wifi", "Normal"]);
plt.title("Wifi - Distribution");

<h3 style='background-color:yellow'>
Mobiles with wifi and without wifi are almost same in count.
    <br> Discrete variable.
</h3>

## <center> Feature relations</center>

### Battery power vs. Price range

In [ ]:
df_dummy=df_train.copy()

In [ ]:
#Function for segregating battery power
def find_range(val):
    if val<=1000:
        return "low"
    elif val >1000 and val<=1500:
        return "Mid"
    elif val > 1500 :
        return "High"

In [ ]:
df_dummy['battery_range']=df_dummy.battery_power.apply(find_range)

In [ ]:
sns.countplot(data=df_dummy,x="battery_range",hue="price_range");
plt.legend(loc='center',bbox_to_anchor=(1.1, 0.5),title="Price range");
plt.title("Battery range vs Price range");

<h3 style='background-color:yellow'>
Battery power < 1000 considering as low. Having more number of phones in Price range 0.<br>
Battery power 1000-1500 considering as Mid. Having all the price ranges in almost same count.<br>
Battery power >1500 considering as High.having more number of phones in Price range 3. <br>
</h3>

In [ ]:
sns.relplot(data=df_train,x="battery_power",y="talk_time");
plt.title("Battery power vs. Talktime");

<h3 style='background-color:yellow'>
There is no direct relation between battery power and talk time in the dataset. <br>
There are mobiles with less battery power with more talk time.
</h3>

In [ ]:
sns.relplot(data=df_train,x="int_memory",y="price_range");
plt.title("Internal memory vs Price range");

<h3 style='background-color:yellow'>
   Mobiles having all ranges of internal memory, lies in all price ranges. <br>
    There are mobiles with less memory having high price range and vice versa.
    </h3>

## <center>Training the model and Prediction </center>

In [ ]:
df_input=df_train.copy()

In [ ]:
def update_df(df_inp):
    categorical_cols=["fc","int_memory","n_cores","pc"]
    for col in categorical_cols:
        print ("Creating categorical columns for ", col)
        df_inp=pd.concat([df_inp,pd.get_dummies(df_inp[col],prefix=col)],axis=1)
        df_inp.drop([col],axis=1,inplace=True)
        print ("Deleted original col ", col)
    return df_inp

In [ ]:
df_input=update_df(df_input)

In [ ]:
x=df_input.drop('price_range',axis=1)
y=df_input.price_range

In [ ]:
def standardize_df(df_inp):
    cont_variables=['battery_power','clock_speed','m_dep','mobile_wt','px_height','px_width','ram','sc_h','sc_w','talk_time']
    for var in cont_variables:
        df_inp[var]=standardize(df_inp[var])
        print ("Standardized feature : ", var)
    return df_inp

In [ ]:
x=standardize_df(x)

In [ ]:
def get_score_data(x,y):
    score_data={
        "LogisticRegression":cross_val_score(LogisticRegression(),x,y),
        "SVM": cross_val_score(SVC(),x,y),
        "RandomForest": cross_val_score(RandomForestClassifier(),x,y),
        "DecisionTree": cross_val_score(DecisionTreeClassifier(),x,y),
        "XGradientBoosting": cross_val_score(XGBClassifier(),x,y),
        "KNN": cross_val_score(KNeighborsClassifier(),x,y)    
    }
    df_score_data = pd.DataFrame(data=score_data)
    print ("Cross validation score:\n ", df_score_data)
    print ("Mean score:\n", df_score_data.mean().sort_values(ascending=False))
    

In [ ]:
get_score_data(x,y)

<h3 style='background-color:yellow'>
Logistic Regression model got highest score.
</h3>

In [ ]:
print ("Train Test splitting..")
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=1)

In [ ]:
print ("Fitting Logistic Regression..")
lr=LogisticRegression()
lr.fit(x_train,y_train)

In [ ]:
print ("Predicting target variable..")
y_pred = lr.predict(x_test)

In [ ]:
print ("Confusion matrix")
confusion_matrix(y_test,y_pred,)

In [ ]:
plt.figure(figsize=(12,7))
sns.heatmap(confusion_matrix(y_test,y_pred),annot=True,cmap='GnBu',fmt="0.2f");
plt.title("Confusion Matrix");

<h3 style='background-color:yellow'>
    130,134,143,139 values are able to predict correctly as 0,1,2,3. <br>
    7 values predicted as 0 which is actually 1. <br>
    5 values predicted as 1 which is actually 0. <br>
    8 values predicted as 2 which is actually 1. <br>
    9 values predicted as 2 which is actually 3. <br>
    11 values predicted as 3 which is actually 2. <br>
    </h3>

In [ ]:
print ("Classification report:")
print (classification_report(y_true= y_test,y_pred=y_pred))

Precision tp/(tp+fp) : measures the ability of a classifier to identify only the correct instances for each class. <br>
Recall tp/(tp+fn) : is the ability of the classifier to find all correct instances per class. <br>
F1 Score = 2 * (precision * recall)/ (precision + recall) <br>
F1 score is considered a better indicator of the classifier's performance than the regular accuracy measure.<br>
Support is the number of actual occurances of the class in the test data set.

In [ ]:
df_dummy=pd.DataFrame()
df_dummy['test']=pd.Series(y_test.values)
df_dummy['pred']=pd.Series(y_pred)

In [ ]:
sns.swarmplot(data=df_dummy,x="test",y='pred');
plt.title("Mapping of Test data to Predicted data");

## <center> Test file prediction</center>

In [ ]:
df_test=pd.read_csv('../input/mobile-price-classification/test.csv')
print ("Test file loaded.")
print ("Shape of test file: ", df_test.shape)
print ("Sample test file")
df_test.head()

In [ ]:
del([df_dummy,df_input])

In [ ]:
df_input=df_test.copy()

In [ ]:
print ("Test file features: ", df_input.columns)

In [ ]:
df_input=update_df(df_input)

In [ ]:
x=df_input.drop('id',axis=1)

In [ ]:
x=standardize_df(x)

In [ ]:
y_pred=lr.predict(x)

In [ ]:
print ("Appending predicted price range to Test file..")
df_test["PriceRangePredicted"]=pd.Series(y_pred)

In [ ]:
df_test

In [ ]:
print ('Exporting predicted results to => working/Exporting to predicted.csv')
df_test.to_csv('./predicted.csv',index=False)